### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [21]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.core.matrix.stats :as stat]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [82]:
;; lazy mode
;(load-large-dataset "../../data-sorted-cleaned/data-CRSP-sorted-cleaned.csv" "main" add-aprc-by-date)
;(set-main "main")

;; non-lazy mode
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio （Go back here everytime you want to restart.）

In [83]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

In [84]:
(keys (deref available-tics))

("AAPL" "F" "IBM")

### Rate of Change (ROC) strategy

ROC = (Closing price - Closing n periods ago) / Closing n periods ago * 100

We'll trade according to the following rules:
- **Buy signal**: when ROC crosses the lower threshold (e.g. -30)
- **Sell signal**: when ROC crosses the upper threshold (e.g. +30

In [85]:
(get-prev-n-days :PRC 10 "AAPL")

({:date "1980-12-15", :PRC "27.3125"})

In [86]:
(count (get-prev-n-days :PRC n "AAPL"))

1

In [87]:
(get (first (get-prev-n-days :PRC n "AAPL")) :PRC)

"27.3125"

In [88]:
(def n 10)
(def lower-threshold -30)
(def upper-threshold 30)
    
(while (< (compare (get-date) "1981-12-31") 0)
    (do
        (if (> (count (get-prev-n-days :PRC n "AAPL")) (- n 1))
            (let [old-price (Double/parseDouble (get (first (get-prev-n-days :PRC n "AAPL")) :PRC)) ;; get price n days agao
                 curr-price (Double/parseDouble (get (last (get-prev-n-days :PRC n "AAPL")) :PRC))] ;; get today's price
                (if (not= old-price 0)
                    (let [roc (* (/ (- curr-price old-price) old-price) 100)]
                        ;(println roc)
                        (if (< roc lower-threshold)
                            (order "AAPL" 1 :print false)) ; buy signal
                        (if (> roc upper-threshold)
                            (order "AAPL" -1 :print false)) ; sell signal
                    ))
                (update-eval-report (get-date))))
        (next-date)
    ))  
(end-order)

[]

### Below: to-change

In [90]:
(doseq [[ticker] (deref portfolio)]
    (if (not= ticker :cash)      
      (order ticker 0 :remaining true)
    )
  )

The order request 1981-12-31 | AAPL | 0 fails.
Failure reason: No such date or ticker in the dataset.


nil

In [32]:
;(pprint/print-table (take 200 (deref portfolio-value)))
(take 10 (deref order-record))

()

### Check portfolio record

In [89]:
;; view final portfolio
(view-portfolio)


| :asset |  :price |   :aprc | :quantity | :tot-val |
|--------+---------+---------+-----------+----------|
|   cash |     N/A |     N/A |       N/A |  1088.79 |
|   AAPL | 22.1875 | 24.3827 |        -3 |   -73.15 |


nil

In [91]:
;; view portfolio value and return
(view-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage | :margin |
|------------+------------+------------+----------+-------+-----------+---------|
| 1980-12-15 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-30 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1980-12-31 |   $1000.37 |      0.00% |    0.02% | $0.00 |      0.00 |   0.00% |
| 1981-01-02 |   $1000.04 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-01-05 |   $1000.96 |      0.04% |    0.04% | $0.00 |      0.00 |   0.00% |
| 1981-01-06 |   $1002.67 |      0.07% |    0.12% | $0.00 |      0.00 |   0.00% |
| 1981-01-07 |   $1004.28 |      0.07% |    0.19% | $0.00 |      0.00 |   0.00% |
| 1981-01-08 |   $1005.03 |      0.03% |    0.22% | $0.00 |      0.00 |   0.00% |
| 1981-01-09 |   $1003.10 |     -0.08% |    0.13% | $0.00 |      0.00 |   0.00% |
| 1981-01-12 |   $1003.32 |      0.01% |    0.14% | $0.00 |      0.00 |   0.00% |


nil

### Generate evaluation report

In [92]:
(eval-report 434)
;; (update-eval-report)
;; (count(deref eval-report-data))
;; (deref dailyret-record)


|      :date | :tot-value |    :vol |  :r-vol |  :sharpe | :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+----------+-----------+---------+---------------|
| 1980-12-29 |      $1000 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $0 |        0.0000 |
| 1980-12-30 |      $1000 | 0.0093% | 0.0093% |  1.7321% |   1.7321% |      $0 |      100.0000 |
| 1980-12-31 |      $1000 | 0.0071% | 0.0071% |  0.2753% |   0.2753% |      $0 |        0.0000 |
| 1981-01-02 |      $1000 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $0 |        0.0000 |
| 1981-01-05 |      $1000 | 0.0177% | 0.0177% |  2.3463% |   2.3463% |      $0 |      100.0000 |
| 1981-01-06 |      $1002 | 0.0314% | 0.0314% |  3.6894% |   3.6894% |      $0 |      100.0000 |
| 1981-01-07 |      $1004 | 0.0345% | 0.0345% |  5.3805% |   5.3805% |      $1 |      100.0000 |
| 1981-01-08 |      $1005 | 0.0320% | 0.0320% |  6.8064% |   6.8064% |      $1 |      100.0000 |
| 1981-01-09 |      $1003 | 0

| 1981-04-28 |      $1007 | 0.0501% | 0.0489% |  6.4547% |   6.6136% |      $2 |      201.6032 |
| 1981-04-29 |      $1007 | 0.0498% | 0.0480% |  6.8954% |   7.1624% |      $2 |      201.6032 |
| 1981-04-30 |      $1007 | 0.0497% | 0.0458% |  6.3071% |   6.8362% |      $2 |      201.6032 |
| 1981-05-01 |      $1007 | 0.0494% | 0.0456% |  6.3439% |   6.8641% |      $2 |      201.6032 |
| 1981-05-04 |      $1007 | 0.0491% | 0.0457% |  6.5850% |   7.0809% |      $2 |      201.6032 |
| 1981-05-05 |      $1007 | 0.0488% | 0.0447% |  6.7604% |   7.3917% |      $2 |      201.6032 |
| 1981-05-06 |      $1008 | 0.0487% | 0.0453% |  7.6127% |   8.1864% |      $2 |      201.6032 |
| 1981-05-07 |      $1008 | 0.0485% | 0.0452% |  7.2234% |   7.7594% |      $2 |      201.6032 |
| 1981-05-08 |      $1007 | 0.0483% | 0.0447% |  6.9779% |   7.5330% |      $2 |      201.6032 |
| 1981-05-11 |      $1008 | 0.0481% | 0.0441% |  7.7071% |   8.4018% |      $2 |      201.6032 |
| 1981-05-12 |      $1008 | 0.

| 1981-08-27 |      $1020 | 0.0476% | 0.0437% | 18.1752% |  19.7902% |      $6 |      200.7140 |
| 1981-08-28 |      $1018 | 0.0478% | 0.0447% | 16.7454% |  17.8843% |      $6 |      200.7140 |
| 1981-08-31 |      $1018 | 0.0476% | 0.0415% | 16.7944% |  19.2832% |      $6 |      200.7140 |
| 1981-09-01 |      $1016 | 0.0479% | 0.0444% | 15.0394% |  16.2140% |      $5 |      200.7140 |
| 1981-09-02 |      $1016 | 0.0478% | 0.0425% | 14.5820% |  16.3968% |      $5 |      200.7140 |
| 1981-09-03 |      $1017 | 0.0479% | 0.0435% | 15.9413% |  17.5371% |      $5 |      200.7140 |
| 1981-09-04 |      $1018 | 0.0478% | 0.0426% | 16.3992% |  18.3973% |      $6 |      200.7140 |
| 1981-09-08 |      $1019 | 0.0477% | 0.0411% | 17.2648% |  20.0242% |      $6 |      200.7140 |
| 1981-09-09 |      $1019 | 0.0476% | 0.0405% | 17.3135% |  20.3527% |      $6 |      200.7140 |
| 1981-09-10 |      $1018 | 0.0475% | 0.0405% | 17.1881% |  20.1327% |      $6 |      200.7140 |
| 1981-09-11 |      $1019 | 0.

| 1981-12-29 |      $1016 | 0.0463% | 0.0399% | 15.6506% |  18.1537% |      $5 |      178.8218 |
| 1981-12-30 |      $1015 | 0.0463% | 0.0404% | 14.5561% |  16.6881% |      $5 |      178.8218 |


nil

In [9]:
(take 1 (deref eval-report-data))

({:date "1981-02-26", :tot-val "$1000", :vol "0.00%", :sharpe "0.00%", :pnl-pt "$0"})

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)